# PROJET DE PYTHON
# THEME : ANALYSE DE LA SATISFACTION DES UTILISATEURS DE LAPTOPS : CAS DES LAPTOPS DELL
# Auteurs:
* MECHIDE SERINE
* METHAFE KUITE SORELLE LOVELINE
* NKAMENI DANIEL
* POUANI EMAPI HORNELLA JOÊLLE

# SCRAPING DES DONNEES SUR LES LAPTOPS DELL

Le but de cette partie est de scraper tous les informations (caractéristiques et commentaires) sur les machines proposées sur le site officiel de Dell France au 10 Dcembre 2021

## SCRAPING DES COMMENTAIRES SUR LES LAPTOPS DELL

Pour scraper les commentaires de chaque machine, il incombe d'ouvrir individuellement les pages de chacune d'elles. Les liens vers la page détaillée de chaque machine sont obtenu à l'aide d'un premier craping. Les pages détaillées des machines sont dynamiques et chargées avec Java Script. Les méthodes de scraping basic vu en cours ne suffiront donc pas dans ce cas. Nous utilisons donc dans la suite le package Selenium adapt au scraping des pages dynamique.

Le pilote Web utilisé est Chrome. l'étape suivante est donc de configurer ce pilote. On définit également un temps d'attente necessaire au chargement de chaque page. Nous avons défini un temps d'attente de 5 secondes.

In [3]:
# Importation des packages

from bs4 import BeautifulSoup 
from selenium import webdriver
import requests  
import re
import time  
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains 
from joblib import Parallel, delayed 
import itertools
import urllib
import bs4
import pandas as pd
from urllib import request

In [ ]:
## Installer chrome driver
driver_path = 'C:/Users/nkame/anaconda3/chromedriver.exe'
opt = webdriver.ChromeOptions()
opt.add_experimental_option('w3c', False)
driver = webdriver.Chrome(executable_path=driver_path,options=opt)


##Initialiser le driver de Selenium
def setupDriver(url, waiting_time = 5):
	driver = webdriver.Chrome(options=opt)
	driver.get(url)
	time.sleep(waiting_time) 
	return driver

La page de chaque machine affiche les premiers commentaires de cette macine en bas de page. Pour les machines dontles commentaires ne tiennent pas sur une page, il faut retrouver l'url de la page suivante et l'utiliser pour acceder aux commentaires suivants. 

Attention: Pour la première page de commentaires comme les page suivante, il est important d'entrer exactement l'url de la section commentaire sinon l'extraction n'aura pas lieu.

Le code suivant permet de charger chaque page detaillée et recupérer le lien de la page des commentaires

In [ ]:
## Charger la page de chaque machine
def getJSpage(url):
	driver = setupDriver(url)
	html = driver.page_source
	driver.close()
	return BeautifulSoup(html, features="lxml")

Une fois la page des commentaires ouverte, il incombe de déterminer la classe qui sera utilisée pour l'extraction. l'inspection de la page revèle qu'il s'agit d'un objet "div" de classe "bv-content-summary-body-text". 

Dans cette classe, sont présents les commentaires des clients ainsi que ceux de Dell. Nous allons par la suite crire une fonction qui suprime les commentaires de Dell.

Rappelonsque les commentaires de certaines machines peuvent se retrouver sur plus d'une page et doivent être pris en compte. La fonction "getNextPage" dans le code suivant permet donc de recuperer l'url de la page suivante dans la page actuelle afin de pouvoir s'y rendre une fois le scraping sur la page actuelle terminé.

La fonction de recupération des commentaires d'une machine est conçue de façon à fonctionner de façon totalement autonome et adaptative en fonction du nombre de pages de commentaires disponibles pour cette machine. En effet, la fonction recupère les commentaires sur une page, vérifie s'il y a une page suivante et si oui passe à celle-ci et recupère les commentaires aussi. Elle s'arrête quand toutes les pages de commentaires ont t visit et scrapé. La structure du site de Dell est assez bizarre mais nous avons réussi à pouvoir implementer cette fonction.

In [ ]:
##Recuperer l'ensemble des commentaires

# Fonction qui recupère les commentaires d'une page
def getPageComments(soupage):
    cpage = soupage.findAll('div', {'class': 'bv-content-summary-body-text'})
    pagecom = [x.text.strip() for x in cpage]
    return pagecom

# Fonction qui recupère l'URL de la prochaine page de commentaires
def getNextPage(soupage):
    nextPage = soupage.findAll('a', {'class': 'bv-content-btn bv-content-btn-pages bv-content-btn-pages-last bv-focusable bv-content-btn-pages-active'})
    return nextPage

# Fonction de recupration des commentaires pour une machine
def getcomments(NCom,Machinelink):
    PageM = getJSpage(Machinelink)
    Pagecom = []
    Pagecom = Pagecom + getPageComments(PageM)
    nextPage = getNextPage(PageM)
    
    # Vérification de l'existence d'une page suivnate
    while(len(nextPage) != 0):
        PageM2 = getJSpage(nextPage[0]["href"])
        Pagecom = Pagecom + getPageComments(PageM2)
        nextPage = getNextPage(PageM2)
    
    # Création des indices (numéro de commande unique pour chaque machine) et de la fameuse base de données
    MIndex = [NCom]*len(Pagecom)
    comments = pd.DataFrame()
    comments["Comments"] = Pagecom
    comments.index = MIndex
    return comments

Une fois les commentaire récupérés, il faut supprimier les commentaires de Dell. Le souci est que tous les commentaires (clients et Dell) sont dans les mêmes balises et classes. Il est clair qu'utiliser les balises ou les classes pour les séparer allait être très compliqué. Nous avons donc analysé les commentaires de Dell et nous avons remarqu qu'ils avaient tous une signature éléctronique contenant "@Dell" ou "@ Dell".

Ceci nous a données l'idée de scraper tous les commentaires et ensuite de supprimer tous ceux contenant @Dell ou @ Dell

In [ ]:
# Suppression des commentaires de DELL
def DelDellComments(Comments):
    CleanComments = Comments[[(('@Dell' not in str(X))&('@ Dell' not in str(X))) for X in Comments["Comments"]]]
    return CleanComments

Avec toutes ces fonctions, nous pouvons à prsent écrire la fonction de récupérations de tous les commentaires des machines disponibles sur le site de Dell. Cette fonction prend en entrée un Data Frame conténant les codes de commande (identificateurs unique de chaque machine) et les url de toutes les machines et renvoie un Data Frame de commenatires

In [ ]:
# Création de la fonction de recupération de tous les commentaires
def getAllComments(Machines):
    AllComments = pd.DataFrame()
    for i in range(len(Machines)):
        Com = getcomments(Machines.iloc[i,0],Machines.iloc[i,1])
        AllComments = pd.concat([AllComments,Com],axis=0)
    AllComments = DelDellComments(AllComments)
    return AllComments

Le bloc de code suivant permet de récupérer les url de toutes les machines à l'aide d'un scraping classique sur le site de de Dell. Ceci permettra de pouvoir construire le Data Frame néccessaire à la récupération des commentaires comme décrit plus haut. 

In [ ]:
# Catégories de machines et nombre de pages sur le site de Dell

list_ref_categ = [["https://www.dell.com/fr-fr/shop/ordinateurs-portables-dell/sr/laptops/inspiron-laptops",6] , ["https://www.dell.com/fr-fr/shop/ordinateurs-portables-dell/sr/laptops/xps-laptops",2], ["https://www.dell.com/fr-fr/shop/ordinateurs-portables-dell/sr/laptops/g-series",2] , ["https://www.dell.com/fr-fr/shop/ordinateurs-portables-dell/sr/laptops/alienware-laptops",3 ]]

In [ ]:
# Fonction de récupérateion des codes des dmachines et les url de toutes les machines

def url (url):
  el=[]
  codess=[]
  for i  in range (1,int(url[1])+1) :
    urlel= url[0] + "?page=" + str(i) 
    req = request.Request(urlel,headers={'User-Agent': 'Mozilla/5.0'})    
    html = request.urlopen(req).read()
    pagefille= bs4.BeautifulSoup(html,"lxml")
    elementss = pagefille.find_all('div', class_ = "ps-image ps-product-image")  # elementss est une liste dont chaque élément contient une balise de type "a" contenant l'url de la machine considérée
    code = pagefille.find_all('div', class_ = "ps-compare") # code est une liste dont chaque élément contient une balise de type "label" contenant le code de commande d'une machine
    elementss_ref= [element.a['href'] for element in elementss]
    el = el + elementss_ref
    codes=[cod.label['oc'] for cod in code ]
    codess = codess + codes
  return el, codess

In [ ]:
# Récupération proprement dite des codes machines et url

elements=[]
codes=[]

for i in range(len(list_ref_categ)):
  el,codess = url(list_ref_categ[i])
  elements = elements + el
  codes = codes + codess

useful_urls = ["https:" + str(raw_url) + "#ratings_section" for raw_url in elements]

In [ ]:
# Création du Data Frame contenant toutes les machines

All_machines = pd.DataFrame({'code_commande': codes, 'url_ordi':useful_urls}).drop_duplicates()

A ce stade, nous pouvons enfin récupérer les commentaires de toutes les machines sur le site de Dell France...

In [ ]:
# Webscraping des commentaires hahaha
# NB: La fonction utilisée ici supprime aussi les commentaire de Dell et garde juste les commentaires clients

All_comments = getAllComments(All_machines)

In [ ]:
All_comments

Pour la suite des analyse, nous enrégistrons ces commentaires dans un fichier Excel car le scraping prend environ 1h30 et nous ne pourrons pas exécuter le code à chaque fois.

In [ ]:
All_comments.to_excel("C:/Users/nkame/Desktop/ND/COURS ET TD/PYTHON/PROJET/Commentaires.xlsx", header = True)

## SCRAPING DES cra$ SUR LES LAPTOPS DELL